Working off of https://levelup.gitconnected.com/deploy-simple-and-instant-online-quizzes-with-jupyter-notebook-tools-5e10f37da531

In [1]:
from ipywidgets import widgets, Layout, Box, GridspecLayout

In [10]:
def create_multipleChoice_widget(description, options, correct_answer, img_filepath = None):
    if correct_answer not in options:
        options.append(correct_answer)
    
    correct_answer_index = options.index(correct_answer)
    
    radio_options = [(words, i) for i, words in enumerate(options)]
    
    alternativ = widgets.RadioButtons(
        options = radio_options,
        description = '',
        disabled = False,
        indent = False,
        align = 'center',
    )
    
    description_out = widgets.Output(layout=Layout(width='auto'))
    
    if img_filepath:
        image = open(img_filepath, 'rb').read()
        image_out = widgets.Image(value=image, 
                                  format='png', 
                                  width='500px', 
                                  height='auto',
                                 )
    
    with description_out:
        print(description)
        
    feedback_out = widgets.Output()

    def check_selection(b):
        a = int(alternativ.value)
        if a==correct_answer_index:
            s = '\x1b[6;30;42m' + "correct" + '\x1b[0m' +"\n"
        else:
            s = '\x1b[5;30;41m' + "try again" + '\x1b[0m' +"\n"
        with feedback_out:
            feedback_out.clear_output()
            print(s)
        return
    
    check = widgets.Button(description="check")
    check.on_click(check_selection)
    
    if img_filepath:
            return widgets.VBox([description_out, 
                         image_out,
                         alternativ, 
                         widgets.HBox([check]), feedback_out], 
                        layout=Layout(display='flex',
                                     flex_flow='column',
                                     align_items='stretch',
                                     width='auto')) 
    
    return widgets.VBox([description_out, 
                         alternativ, 
                         widgets.HBox([check]), feedback_out], 
                        layout=Layout(display='flex',
                                     flex_flow='column',
                                     align_items='stretch',
                                     width='auto')) 

In [11]:
create_multipleChoice_widget("a or b", ['a','b'], 'b')

In [12]:
create_multipleChoice_widget("a or b", ['a','b'], 'b', 'test.png')

- Alright, seems to be working
    - Next, we need to come up with a way to keep track of results instead of saying "try again" vs. "correct"

- We can define a global variable to do this
    - Also need to delete check button after it's answered

In [41]:
quiz_results = []

def create_multipleChoice_widget(description, options, correct_answer, img_filepath = None):
    if correct_answer not in options:
        options.append(correct_answer)
    
    correct_answer_index = options.index(correct_answer)
    
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    options_w_letters = [f'{alphabet[i]}) {option}' for i, option in enumerate(options)]
    
    radio_options = [(words, i) for i, words in enumerate(options_w_letters)]
    
    alternativ = widgets.RadioButtons(
        options = radio_options,
        description = '',
        disabled = False,
        indent = False,
        align = 'center',
    )
    
    description_out = widgets.Output(layout=Layout(width='auto'))
    
    if img_filepath:
        image = open(img_filepath, 'rb').read()
        image_out = widgets.Image(value=image, 
                                  format='png', 
                                  width='500px', 
                                  height='auto',
                                 )
    
    with description_out:
        print(description)
        
    feedback_out = widgets.Output()

    def check_selection(b):
        a = int(alternativ.value)
        if a==correct_answer_index:
            s = '\x1b[6;30;42m' + "correct" + '\x1b[0m' +"\n"
        else:
            correct_answer_letter = alphabet[correct_answer_index]
            s = '\x1b[5;30;41m' + f"incorrect, the correct answer was {correct_answer_letter}" + '\x1b[0m' +"\n"
        with feedback_out:
            print(s)
            global quiz_results
            tuple_results = (s, a==correct_answer_index)
            quiz_results.append(tuple_results)
            alternativ.close()
            check.layout.display = 'none'
        return
    
    check = widgets.Button(description="check")
    check.on_click(check_selection)
    
    check_out = widgets.HBox([check])
    
    if img_filepath:
            return widgets.VBox([description_out, 
                         image_out,
                         alternativ, 
                         widgets.HBox([check]), feedback_out], 
                        layout=Layout(display='flex',
                                     flex_flow='column',
                                     align_items='stretch',
                                     width='auto')) 
    
    return widgets.VBox([description_out, 
                         alternativ, 
                         check_out, 
                         feedback_out], 
                        layout=Layout(
                            display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            width='auto')) 

In [42]:
create_multipleChoice_widget("a or b", ['a','b'], 'b', 'test.png')

- Alright, looks like we got the essential parts
    - We'll start a new version in the next notebook